In [ ]:
import datetime
import logging

import numpy as np
from matplotlib import pyplot as plt

import ocean_navigation_simulator
from ocean_navigation_simulator.environment.ArenaFactory import ArenaFactory
from ocean_navigation_simulator.environment.PlatformState import (
    SpatioTemporalPoint,
)
from ocean_navigation_simulator.problem_factories.Constructor import (
    Constructor,
)
from ocean_navigation_simulator.utils.misc import set_arena_loggers

set_arena_loggers(logging.DEBUG)

# Set-Up for Simulation

## Platform Speed, Forecasted and true currents

In [ ]:
print(ocean_navigation_simulator.__file__)

In [ ]:
# Forecast System that we can use:
# - HYCOM Global (daily forecasts, xh resolution, 1/12 deg spatial resolution)
# - Copernicus Global (daily forecasts for 5 days out, xh resolution, 1/12 deg spatial resolution)
# - NOAA West Coast Nowcast System (daily nowcasts for 24h, xh resolution, 10km spatial resolution)
# Note: if you switch it, you need to delete the old FC files, otherwise it'll use those.
max_speed_of_platform_in_meter_per_second = 0.2
forecast_system_to_use = "noaa"  # either of ['HYCOM', 'Copernicus', 'noaa']
folder_for_forecast_files = 'data/noaa_forecast_files/'

# Currently true currents are set to Copernicus
true_ocean_current_dict = {
            "field": "OceanCurrents",
            "source": "opendap",
            "source_settings": {
                "service": "copernicus",
                "currents": "total",
                "USERNAME": "mmariuswiggert",
                "PASSWORD": "tamku3-qetroR-guwneq",
                "DATASET_ID": "cmems_mod_glo_phy_anfc_merged-uv_PT1H-i",
            }}

# Configs for simulator
simulation_timeout = 3600*24 *5  # 5 days
arena_config = {
    "casadi_cache_dict": {
        "deg_around_x_t": 0.3,
        "time_around_x_t": 86400.0,
    },  # This is 24h in seconds!
    "platform_dict": {
        "battery_cap_in_wh": 400.0,
        "u_max_in_mps": max_speed_of_platform_in_meter_per_second,
        "motor_efficiency": 1.0,
        "solar_panel_size": 1.0,
        "solar_efficiency": 0.2,
        "drag_factor": 675.0,
        "dt_in_s": 600.0,
    },
    "use_geographic_coordinate_system": True,
    "spatial_boundary": None,
    "ocean_dict": {
        "region": "Region 1",  # This is the region of northern California
        "hindcast": true_ocean_current_dict,
        "forecast": {
            "field": "OceanCurrents",
            "source": "forecast_files",
            "source_settings": {"source": forecast_system_to_use,
                                "folder": folder_for_forecast_files,
                                "type": "forecast"},
        },
    },
    "solar_dict": {"hindcast": None, "forecast": None},
    "seaweed_dict": {"hindcast": None, "forecast": None},
}

objectiveConfig = {"type": "nav"}

## Controller Settings

In [4]:
t_max_planning_ahead_in_seconds = 3600*60 # that is 60h

ctrl_config={  'T_goal_in_seconds': t_max_planning_ahead_in_seconds,
              'accuracy': 'high',
              'artificial_dissipation_scheme': 'local_local',
              'ctrl_name': 'ocean_navigation_simulator.controllers.hj_planners.HJReach2DPlanner.HJReach2DPlanner',
              'deg_around_xt_xT_box': 0.3,
              'direction': 'multi-time-reach-back',
              'grid_res': 0.005,
              'n_time_vector': 100,
              'obstacle_dict': {'obstacle_value': 1,
                                'path_to_obstacle_file': 'ocean_navigation_simulator/package_data/bathymetry_and_garbage/bathymetry_distance_res_0.004_max_elevation_0_northern_california.nc',
                                'safe_distance_to_obstacle': 0},
              'progress_bar': True,
              'replan_every_X_seconds': None,
              'replan_on_new_fmrc': True,
               'calc_opt_traj_after_planning': True,
              'use_geographic_coordinate_system': True}

## Mission Setting (Start -> Target Region)

In [5]:
print("current UTC datetime is: ", datetime.datetime.now())

start_time = "2023-05-03T10:00:00+00:00"  # in UTC
OB_point = {"lat": 37.738160, "lon": -122.545469}
HMB_point = {"lat": 37.482812, "lon": -122.531450}

x_0_dict = {"date_time": start_time}
x_0_dict.update(HMB_point)

missionConfig = {
    "target_radius": 0.02,  # in degrees
    "x_0": [x_0_dict],  # the start position and time
    "x_T": OB_point,  # the target position
}

current UTC datetime is:  2023-05-02 19:41:47.723078


In [6]:
point_to_check = SpatioTemporalPoint.from_dict(missionConfig['x_0'][0])
t_interval = [point_to_check.date_time - datetime.timedelta(hours=35),
              point_to_check.date_time + datetime.timedelta(
                  seconds=simulation_timeout
                          + arena_config['casadi_cache_dict'][
                      'time_around_x_t'] + 7200)]

# Note: it connects to C3 and downloads the relevant forecast files to the local repo
ArenaFactory.download_required_files(
        archive_source = arena_config['ocean_dict']['forecast']['source_settings']['source'],
        archive_type = arena_config['ocean_dict']['forecast']['source_settings']['type'],
        download_folder=arena_config['ocean_dict']['forecast']['source_settings']['folder'],
        t_interval = t_interval,
        region= arena_config['ocean_dict']['region'],
        points= [point_to_check])

Starting to connect to c3


Only 2/8 files in the database for noaa, forecast, Region 1 and t_0=2023-05-01 23:00:00+00:00 and t_T=2023-05-09 12:00:00+00:00: 
- nos.wcofs.regulargrid.f003-f072.20230501.t03z.nc
- nos.wcofs.regulargrid.f003-f072.20230502.t03z.nc
100%|██████████| 2/2 [00:00<00:00, 95.51it/s]


['data/noaa_forecast_files/nos.wcofs.regulargrid.f003-f072.20230501.t03z.nc',
 'data/noaa_forecast_files/nos.wcofs.regulargrid.f003-f072.20230502.t03z.nc']

In [7]:
# Step 0: Create Constructor object which contains arena, problem, controller
constructor = Constructor(
    arena_conf=arena_config,
    mission_conf=missionConfig,
    objective_conf=objectiveConfig,
    ctrl_conf=ctrl_config,
    observer_conf={"observer": None},
    timeout_in_sec=simulation_timeout,
    throw_exceptions=False,
)

# Step 1.1 Retrieve problem
problem = constructor.problem

# Step 1.2: Retrieve arena
arena = constructor.arena
observation = arena.reset(platform_state=problem.start_state)
problem_status = arena.problem_status(problem=problem)

# Step 2: Retrieve Controller
controller = constructor.controller

INFO:arena.platform:Platform: Set Dynamics F_x_next Function (0.0s)
INFO:arena.platform:Platform: Update Casadi + Dynamics (7.5s)


# Run Planning on Forecasts and Visualizing the plan if FC are accurate

In [8]:
#% Run first planning
action = controller.get_action(observation=observation)

INFO:arena.controller:HJPlannerBase: Planning because of new forecast (Old: None, New: 2023-05-02 06:00:00+00:00)


FileNotFoundError: [Errno 2] No such file or directory: b'/Volumes/Data/2_Work/2_Graduate_Research/1_Seaweed/OceanPlatformControl/scripts/RealOceanRun/ocean_navigation_simulator/package_data/bathymetry_and_garbage/bathymetry_distance_res_0.004_max_elevation_0_northern_california.nc'

In [ ]:
# Visualize plan if forecast is accurate
ax = controller.plot_reachability_snapshot(
    rel_time_in_seconds=0,
    granularity_in_h=5,
    alpha_color=1,
    time_to_reach=True,
    fig_size_inches=(12, 12),
    plot_in_h=True,
    return_ax=True,
)

# problem.plot(ax=ax)
ax.plot(controller.x_traj[0, :], controller.x_traj[1, :], color='k', label="State Trajectory", linewidth=2)
ctrl_stride = 5
u_vec = controller.contr_seq[0, ::ctrl_stride] * np.cos(controller.contr_seq[1, ::ctrl_stride])
v_vec = controller.contr_seq[0, ::ctrl_stride] * np.sin(controller.contr_seq[1, ::ctrl_stride])
ax.quiver(
    controller.x_traj[0, :-1:ctrl_stride],
    controller.x_traj[1, :-1:ctrl_stride],
    u_vec,
    v_vec,
    color='magenta',
    scale=15,
    angles="xy",
    label="Control Inputs",
)
plt.show()

In [ ]:
# Further animations of the value function
# controller.animate_value_func_3D()
#%
# controller.plot_reachability_snapshot_over_currents(rel_time_in_seconds=0, granularity_in_h=5, time_to_reach=False)
# controller.plot_reachability_animation(time_to_reach=False, granularity_in_h=5, filename="test_reach_animation.mp4")
# controller.plot_reachability_animation(time_to_reach=True, granularity_in_h=5, with_opt_ctrl=True,
#                                     filename="test_reach_animation_w_ctrl.mp4", forward_time=True)

# Run Closed-loop simulation with true currents

In [ ]:
#% Run closed-loop simulation
while problem_status == 0:
    # Get action
    action = controller.get_action(observation=observation)

    # execute action
    observation = arena.step(action)

    # update problem status
    problem_status = arena.problem_status(problem=problem)

In [ ]:
# Visualize the closed-loop trajectory
arena.plot_all_on_map(problem=problem, margin=0.1, vmax=0.5, vmin=0,
                      control_stride=8, control_vec_scale=8,
                      spatial_resolution=1 / 30)

In [ ]:
# Animate closed-loop trajectory
arena.animate_trajectory(margin=0.1,
                         ctrl_scale=8,
                         temporal_resolution=3600,
                         spatial_resolution=1 / 30,
                         vmax=0.5, vmin=0,
                         filename="test.mp4")

took_h = (arena.state_trajectory[-1, 2] - arena.state_trajectory[0, 2]) / 3600
print("Closed-Loop Operation took {} hours".format(took_h))
t_start = datetime.datetime.fromisoformat(missionConfig['x_0'][0]['date_time'])

In [ ]:
# Visualize the true currents and the forecasted currents (without trajectory)
def add_prob(ax, time):
    problem.plot(ax=ax)

arena.ocean_field.hindcast_data_source.animate_data(
    output="hindcast.mp4",
    x_interval=[-123, -122],
    y_interval=[37, 38],
    t_interval=[t_start, t_start + datetime.timedelta(hours=60)],
    vmax=0.4, vmin=0.0,
    add_ax_func=add_prob
)
arena.ocean_field.forecast_data_source.animate_data(
    output="forecast.mp4",
    x_interval=[-123, -122],
    y_interval=[37, 38],
    t_interval=[t_start, t_start + datetime.timedelta(hours=60)],
    vmax=0.4, vmin=0.0,
    add_ax_func=add_prob
)